In [1]:
#Libraries
from copy import deepcopy
import time
import random

In [2]:
# Configuration constants
NUM_ROWS = 6
NUM_COLS = 7

In [3]:
# Creates / Check / Save states
class State:
    def __init__(self):
        # initialize the board info here and any additional variables
        self.board = [[0]*NUM_COLS for i in range(NUM_ROWS)] # [[0,0,0,...], [0,0,0,...], ...] board initial state (all zeros)
        self.column_heights = [NUM_ROWS - 1] * NUM_COLS # [5, 5, 5, 5, 5, 5, 5] useful to keep track of the index in which pieces should be inserted
        self.available_moves = list(range(7)) # [0, 1, ..., 6] list of playable columns (not full)
        self.player = 1
        self.winner = -1 # -1 - no winner (during game); 0 - draw; 1- player 1; 2 - player 2
        
    def check_line(self, n, player, values):
        num_pieces = sum(list(map(lambda val: val == player, values)))
        # code above is equivalent to
        # num_pieces = 0
        # for val in values:
        #     if val == player:
        #         num_pieces += 1
        if n == 4:
            return num_pieces == 4
        if n == 3:
            num_empty_spaces = sum(list(map(lambda val: val == 0, values)))
            return num_pieces == 3 and num_empty_spaces == 1
    
    # calculates the number of lines with 4 pieces (horizontal, vertical, diagonal) of a given player.
    # calculates the number of sets of 4 consecutive spots that have three pieces of the player followed by an empty spot, i.e., that are possibilities to win the game.
    def count_lines(self, n, player):
        num_lines = 0
        for row in range(NUM_ROWS):
            for col in range(NUM_COLS):
                # checks vertical line
                if col < NUM_COLS - 3 and self.check_line(n, player, [self.board[row][col], self.board[row][col+1], self.board[row][col+2], self.board[row][col+3]]):
                    num_lines += 1
                # checks horizontal line
                if row < NUM_ROWS - 3 and self.check_line(n, player, [self.board[row][col], self.board[row+1][col], self.board[row+2][col], self.board[row+3][col]]):
                    num_lines += 1
                # checks upper diagonal line
                if row < NUM_ROWS - 3 and col < NUM_COLS - 3 and self.check_line(n, player, [self.board[row][col], self.board[row+1][col+1], self.board[row+2][col+2], self.board[row+3][col+3]]):
                    num_lines += 1
                # checks lower diagonal line
                if row < NUM_ROWS - 3 and col > 3 and self.check_line(n, player, [self.board[row][col], self.board[row+1][col-1], self.board[row+2][col-2], self.board[row+3][col-3]]):
                    num_lines += 1
        return num_lines
    
    # assigns 2 points to each player piece in the center column of the board (column 4) and 1 point to each piece in the columns around it (columns 3 and 5).
    def central(self, player):
        points = 0
        for row in range(NUM_ROWS):
            if self.board[row][4] == player: # center column
                points += 2  
            if self.board[row][3] == player: # around center column
                points += 1 
            if self.board[row][5] == player: # around center column
                points += 1
        return points
    
    def move(self, column): 
        # function that performs a move given the column number and returns the new state
        #--------------------------------------------------#
        state_copy = deepcopy(self)
        
        height = state_copy.column_heights[column]
        state_copy.column_heights[column] = height
        state_copy.board[height][column] = self.player
        
        if height == 0:
            state_copy.available_moves.remove(column)
        else:
            state_copy.column_heights[column] = height - 1
        
        state_copy.update_winner() 
        state_copy.player = 3 - self.player # update player turn
        
        return state_copy

    def update_winner(self):
        if self.count_lines(4, 1) > 0:
            self.winner = 1
        elif self.count_lines(4, 2) > 0:
            self.winner = 2
        elif len(self.available_moves) == 0:
            self.winner = 0

In [4]:
# Interface do jogo via terminal

def display_board(board):
        #Print the Connect 4 board in a readable format
        print("\n  1 2 3 4 5 6 7")  # Column numbers
        print(" ---------------")
        for row in board:
            print("|", end="")
            for cell in row:
                if cell == 0:
                    print(" ", end="|")
                elif cell == 1:
                    print("X", end="|")
                elif cell == 2:
                    print("O", end="|")
            print()
        print(" ---------------")

def print_welcome():
    print("\n===== CONNECT 4 =====")
    print("1. Player vs Player")
    print("2. Player vs Computer")
    print("3. Computer vs Computer")
    print("4. Exit")
    print("====================")



In [5]:
# Create game 

class ConnectFourGame:
    def __init__(self, player_1_ai, player_2_ai):
        self.state = State() # initial state
        self.player_1_ai = player_1_ai # store player 1 type (move selection method)
        self.player_2_ai = player_2_ai # store player 2 type (move selection method)
        
    def start(self, log_moves = False):
        self.state = State()
        while True:
            # play the move
            if self.state.player == 1:
                self.player_1_ai(self)
            else:
                self.player_2_ai(self)          
            if log_moves:
                print(self.state.board)
            
            # check the winner and end the game if so
            if self.state.winner != -1:
                break
        # print the winner of the game
        if self.state.winner == 0:
            print("End of game! Draw!")
        else:
            print(f"End of game! Player {self.state.winner} wins!")
    
    def run_n_matches(self, n, max_time = 3600, log_moves = False):
        # utility function to automate n matches execution
        # should return the total distribution of players wins and draws
        start_time = time.time()
        
        results = [0, 0, 0] # [draws, player 1 victories, player 2 victories]
        
        while n > 0 and time.time() - start_time < max_time:
            n -= 1
            self.start(log_moves)
            results[self.state.winner] += 1
            
        print("\n=== Elapsed time: %s seconds ===" % (int(time.time() - start_time)))
        print(f"  Player 1: {results[1]} victories")
        print(f"  Player 2: {results[2]} victories")
        print(f"  Draws: {results[0]} ")
        print("===============================")



In [ ]:
# AI

def execute_random_move(game):
    move = random.choice(game.state.available_moves)
    game.state = game.state.move(move)

def human_player_move(game):
    # Allow a human player to choose a move via console input
    while True:
        try:
            print(f"\nPlayer {game.state.player}'s turn")
            print(f"Available columns: {[col+1 for col in game.state.available_moves]}")
            column = int(input(f"Enter column number (1-{len(game.state.available_moves)}): ")) - 1  # Convert to 0-indexed

            
            if column in game.state.available_moves:
                game.state = game.state.move(column)
                break
            else:
                print("Invalid move! Column is full or out of range.")
        except ValueError:
            print("Please enter a number between 1 and {NUM_COLS}.")

# Aqui tem uma heuristica inventada pelo GPT, a ideia é modificar para MonteCarlo / Decision Tree
def execute_smart_move(game):
    """
    A smarter AI that evaluates potential moves and chooses the best one
    based on simple heuristics:
    1. If can win in one move, make that move
    2. If opponent can win in one move, block that move
    3. Prefer center columns
    4. Otherwise choose a move that maximizes potential winning lines
    """
    available_moves = game.state.available_moves
    current_player = game.state.player
    opponent = 3 - current_player  # In Connect 4, players are 1 and 2
    
    # Check if we can win in this move
    for move in available_moves:
        new_state = game.state.move(move)
        if new_state.winner == current_player:
            game.state = new_state
            return
    
    # Check if opponent would win in their next move and block it
    for move in available_moves:
        # Simulate opponent playing in this position
        test_state = game.state.move(move)
        opponent_state = test_state
        opponent_state.player = opponent  # Change player for evaluation
        
        for opponent_move in opponent_state.available_moves:
            if opponent_move == move:  # Skip the move we just made in simulation
                continue
            potential_state = opponent_state.move(opponent_move)
            if potential_state.winner == opponent:
                # Found a move where opponent could win next turn, block it
                game.state = game.state.move(move)
                return
    
    # Evaluate all potential moves
    best_move = None
    best_score = float('-inf')
    
    for move in available_moves:
        new_state = game.state.move(move)
        # Score the move based on:
        # - Number of potential winning lines (3 in a row with empty space)
        # - Central position control
        score = (new_state.count_lines(3, current_player) * 10 + 
                 new_state.central(current_player) * 2 - 
                 new_state.count_lines(3, opponent) * 8)
        
        if score > best_score:
            best_score = score
            best_move = move
    
    # If still no good move found, prefer central columns
    if best_move is None:
        for preferred in [3, 2, 4, 1, 5, 0, 6]:  # Preference order (center to edges)
            if preferred in available_moves:
                best_move = preferred
                break
    
    # Make the best move
    game.state = game.state.move(best_move)




    



In [ ]:
# PEDRO MONTE CARLO

#root = current game.state
class tree:
        def __init__(self, root):
            self.nodes = [root]
            self.root = root

        def expand(self, node):
            new_node = node.add_child()
            self.nodes.append(new_node)
            return new_node

class node:
    def __init__(self, game):
        self.game = game
        self.visits = 0
        self.wins = 0
        self.children = []

    def isleaf(self):
        if self.children == []:
            return True
        return False
    
    def add_child(self):
        new_game = deepcopy(self.game)
        move = random.choice(game.state.available_moves)
        new_game.state = game.state.move(move)
        child = node(new_game)
        self.children.append(child)
        return child
    
    def updateStats(self, value):
        self.wins += value
        self.visits += 1

def monte_carlo(epochs, arvore, c):
    for i in range(epochs):
        visited = []
        node = arvore.root
        visited.append(node)
        while (not node.isleaf()):
            node = select(node, node.children, c)#todo (formula para selecionar é (vitorias/analizes)+ Constante X (log(visitas pai)/ vistias filho) ** 1/2)            
            # I believe it may be a good Idea to permit multiple variations of c 1, sqr(2) and sqr(3) seem like good opitions for variability and exploration
            visited.append(node) 

        newChild = arvore.expand(node) 
        visited.append(newChild) # add the new node with one visit to the visited list

        value = rollOut(newChild) # simulate the new node from the node selected (use random moves until the game is over and sees who won (may be intersting to explore the possibility of
        ## allowing for different implementations sutch as +1 for victory and 0 for draws, and either 0 or - 1 for losses, could be intersting))


        for node in visited:
            node.updateStats(value) # Not sure about this implementation, what we have to do is update the victory amount for all the nodes that where on the path
            #I was wrong, it is not that complicated, the visited list is reset after eatch attempt


    return bestaction(arvore.root, c) # return the node that has the best value from the childrem of the root (current state of the game)

def select(node, node_children, c):
    highest_value_node = None #todo (formula para selecionar é (vitorias/analizes)+ Constante X (log(visitas pai)/ vistias filho) ** 1/2) 
    highest_value = 0
    for child in node_children:
        if child.wins/child.visits + (c*((math.log(node.visits)/child.visits)**1/2)) > highest_value:
            highest_value = child.wins/child.visits + (c*((math.log(node.visits)/child.visits)**1/2)) 
            highest_value_node = child
    return highest_value_node 

def bestaction(root, c):
    return select(root, root.children, c)

def rollOut(node):
    while node.game.state.winner == -1:
        execute_random_move(node.game)
    return node.game.state.winner

def execute_random_move(game):
    move = random.choice(game.state.available_moves)
    game.state = game.state.move(move)

def execute_monte_carlo_move(game, epochs, c):   #May need to add a player variable to know who is playng and using the algo to calculae victorys
    def execute_monte_carlo_move_aux(game):
        #todo
        next_move = monte_carlo(epochs, tree(node(game)), c)
        game.state = next_move.game.state
    return execute_monte_carlo_move_aux #AFTER THE CURRENT MOVE HAS PASSED THE ENTIRE TREE IS DELETED when using the following implementation

In [7]:
while True:
        print_welcome()
        choice = input("Select game mode: ")
        
        if choice == "1":  # Player vs Player
            game = ConnectFourGame(human_player_move, human_player_move)
            # Override the original board printing to use our nicer display
            original_start = game.start
            def new_start(log_moves=True):
                game.state = game.state.__class__()
                while True:
                    display_board(game.state.board)
                    if game.state.player == 1:
                        game.player_1_ai(game)
                    else:
                        game.player_2_ai(game)
                    
                    if game.state.winner != -1:
                        display_board(game.state.board)
                        break
                
                if game.state.winner == 0:
                    print("End of game! Draw!")
                else:
                    print(f"End of game! Player {game.state.winner} wins!")
            
            game.start = new_start
            game.start()
            
        elif choice == "2":  # Player vs Computer
            # Ask if player wants to be Player 1 or Player 2
            player_choice = ""
            while player_choice not in ["1", "2"]:
                player_choice = input("Do you want to play as Player 1 (X) or Player 2 (O)? (1/2): ")
            
            if player_choice == "1":
                game = ConnectFourGame(human_player_move, execute_smart_move)
                player_symbol = "X"
                computer_symbol = "O"
            else:
                game = ConnectFourGame(execute_smart_move, human_player_move)
                player_symbol = "O"
                computer_symbol = "X"
            
            # Use the same display method as Player vs Player
            def new_start(log_moves=True):
                game.state = game.state.__class__()
                while True:
                    display_board(game.state.board)
                    
                    # Show whose turn it is
                    current_player = "Your" if ((game.state.player == 1 and player_choice == "1") or 
                                               (game.state.player == 2 and player_choice == "2")) else "Computer's"
                    print(f"\n{current_player} turn ({player_symbol if current_player == 'Your' else computer_symbol})")
                    
                    if game.state.player == 1:
                        game.player_1_ai(game)
                    else:
                        game.player_2_ai(game)
                    
                    if game.state.winner != -1:
                        display_board(game.state.board)
                        break
                
                if game.state.winner == 0:
                    print("End of game! Draw!")
                else:
                    winner = "You" if ((game.state.winner == 1 and player_choice == "1") or 
                                       (game.state.winner == 2 and player_choice == "2")) else "Computer"
                    print(f"End of game! {winner} win!")
            
            game.start = new_start
            game.start()
            
        elif choice == "3":  # Computer vs Computer
            # Allow selecting different AI types
            print("\nSelect AI types:")
            print("1. Random vs Random")
            print("2. Smart vs Random")
            print("3. Smart vs Smart")
            ai_choice = input("Enter your choice (1-3): ")
            
            if ai_choice == "2":
                game = ConnectFourGame(execute_smart_move, execute_random_move)
            elif ai_choice == "3":
                game = ConnectFourGame(execute_smart_move, execute_smart_move)
            else:  # Default to random vs random
                game = ConnectFourGame(execute_random_move, execute_random_move)
                
            num_games = int(input("How many games to run? "))
            game.run_n_matches(num_games, 120, True)
            
        elif choice == "4":  # Exit
            print("Thanks for playing!")
            break
            
        else:
            print("Invalid choice. Please try again.")


===== CONNECT 4 =====
1. Player vs Player
2. Player vs Computer
3. Computer vs Computer
4. Exit
Thanks for playing!
